In [10]:
using ExpressionTreeForge

function length(node::ExpressionTreeForge.M_implementation_tree.Type_node{T}) where {T}
    if isempty(node.children)
        return 1
    else
        return sum(length, node.children)
    end
end

function tree_depth(node::ExpressionTreeForge.M_implementation_tree.Type_node{T}) where {T}
    isempty(node.children) && return 1
    return 1 + maximum(tree_depth.(node.children))
end


tree_depth (generic function with 1 method)

In [ ]:
using OptimizationProblems
using ADNLPModels
using Symbolics
using ExpressionTreeForge # A bit outdated so it's not easy to find a compatible version.
using CSV, DataFrames

meta = OptimizationProblems.meta
problem_names = meta[(meta.contype .== :unconstrained) .& (.!meta.has_bounds) .& (meta.nvar .>= 5),:name]

problems = [Meta.parse("OptimizationProblems.ADNLPProblems.$(problem)") 
                    for problem ∈ problem_names];

filename = "../metadata/length_and_depth.csv"
mkpath(dirname(filename))

df = DataFrame(
    :problem => String[],
    :length => Int[],
    :depth => Int[]
)

for pb_expr in problems
    try
        nlp = eval(pb_expr)()     # call the zero-arg constructor
        println("Extracting information of $(nlp.meta.name)")
        n = nlp.meta.nvar
        Symbolics.@variables x[1:n]
        fun = nlp.f(x)
        mtk_tree = Symbolics._toexpr(fun)
        expr_tree_Symbolics = transform_to_expr_tree(mtk_tree)

        tree_length = tree_length(expr_tree_Symbolics)
        tree_depth = tree_depth(expr_tree_Symbolics)
        push!(df, (; problem = nlp.meta.name, tree_length, tree_depth))

        CSV.write(
            filename,
            DataFrame([last(df)]);
            append = isfile(filename) && filesize(filename) > 0,
        )
    catch e
        @info "Failed to extract information on $(nlp.meta.name): $e"
    end
end

In [4]:
using OptimizationProblems
using ADNLPModels

# https://jso.dev/ExpressionTreeForge.jl/dev/tutorial/ Link to ExpressionTreeForge Tutorial

# Data from OptimizationProblems.jl
# Careful, nvar and ncon are the default, they may vary if variable_nvar or variable_ncon are true.
meta = OptimizationProblems.meta
name_problem = "arglina"

# Get the type
nlp = OptimizationProblems.ADNLPProblems.arglina()
# Return the type of problem in terms of programming
type_of_pb = typeof(nlp)
if type_of_pb <: ADNLPModel
    objective_function = nlp.f # this is a function
    # This gives information on how is computed the gradient.
    gradient_backend = typeof(nlp.adbackend.gradient_backend)
end

nlp.meta # also contains more info on the problem, but mainly useless for unconstrained problems

# How to get the expression tree of the function:
# use Symbolics to write it as an Expr
# then ExpressionTreeForge make a tree out of it.
using Symbolics

n = 100
Symbolics.@variables x[1:n] # must be x
fun = nlp.f(x)
mtk_tree = Symbolics._toexpr(fun)
using ExpressionTreeForge # A bit outdated so it's not easy to find a compatible version.
expr_tree_Symbolics = transform_to_expr_tree(mtk_tree)

expr_tree = copy(expr_tree_Symbolics)
element_functions = extract_element_functions(expr_tree)
show(element_functions[2])

*
  50//1
  ^2
    +
      -1
      -
        *
          1//100
          +
            x[1]
            x[10]
            x[100]
            x[11]
            x[12]
            x[13]
            x[14]
            x[15]
            x[16]
            x[17]
            x[18]
            x[19]
            x[2]
            x[20]
            x[21]
            x[22]
            x[23]
            x[24]
            x[25]
            x[26]
            x[27]
            x[28]
            x[29]
            x[3]
            x[30]
            x[31]
            x[32]
            x[33]
            x[34]
            x[35]
            x[36]
            x[37]
            x[38]
            x[39]
            x[4]
            x[40]
            x[41]
            x[42]
            x[43]
            x[44]
            x[45]
            x[46]
            x[47]
            x[48]
            x[49]
            x[5]
            x[50]
            x[51]
            x[52]
            x[53]
            x[54]
         

In [5]:
typeof(expr_tree_Symbolics)


ExpressionTreeForge.M_implementation_tree.Type_node{ExpressionTreeForge.M_abstract_expr_node.Abstract_expr_node}

In [6]:
tree_depth(expr_tree_Symbolics)

9

In [11]:
length(expr_tree_Symbolics)

10404

In [ ]:
using OptimizationProblems
using ADNLPModels
using Symbolics
using ExpressionTreeForge # A bit outdated so it's not easy to find a compatible version.
using CSV, DataFrames


# https://jso.dev/ExpressionTreeForge.jl/dev/tutorial/ Link to ExpressionTreeForge Tutorial

# Data from OptimizationProblems.jl
# Careful, nvar and ncon are the default, they may vary if variable_nvar or variable_ncon are true.
meta = OptimizationProblems.meta
problem_names = meta[(meta.contype .== :unconstrained) .& (.!meta.has_bounds) .& (meta.nvar .>= 5),:name]

problems = [Meta.parse("OptimizationProblems.ADNLPProblems.$(problem)") 
                for problem ∈ problem_names];

filename = "../metadata/length_and_depth.csv"
mkpath(dirname(filename))

df = DataFrame(
    :problem => String[],
    :length => Int[],
    :depth => Int[]
)

for problem in problems
    try
        nlp = eval(problem)()      # call the zero-arg constructor
        println("Extracting information of $(nlp.meta.name)")
        n = nlp.meta.nvar
        Symbolics.@variables x[1:n]
        fun = nlp.f(x)
        mtk_tree = Symbolics._toexpr(fun)
        expr_tree_Symbolics = transform_to_expr_tree(mtk_tree)

        length = tree_length(expr_tree_Symbolics)
        tree_depth = tree_depth(expr_tree_Symbolics)
        push!(df, (; problem = nlp.meta.name, tree_length, tree_depth))

        CSV.write(
            filename,
            DataFrame([last(df)]);
            append = isfile(filename) && filesize(filename) > 0,
        )
    catch e
        @info "Failed to extract information on $(problem): $e"
        @info "Failed to extract information on $(problem): $e"
    end
end












